# Tensorflow Tutorial 2.0

This notebook is a quick introduction to Tensorflow, from the point of view of using Tensorflow via Python notebooks. Tensorflow is an open source framework for **high performance machine learning**, while you don't need Tensorflow to run a logistic regression (although as we'll see, you can easily do so), you'll need something like it to perform the heavy lifting for the types of models.

From a technical standpoint, Tensorflow combines the following:

* the ability to define computations in a symbolic manner and extract derivatives automatically by back-propagation
* the ability to execute these operations on a variety of hardware, including GPUs and TPUs (GPUs are great for general purpose parallization; TPUs are great at multipling tensors together, which happens a lot when training deep networks)


The benefit to you is that this allows you to focus on:

* specifying the model (i.e the architecture of your neural network)
* specifying the loss (e.g cross entropy for classifcation, squared loss for regression etc)
* specifying the mode of training (e.g SGD, Adam etc)

rather than the technical aspects of the implementation of the training procedure. I'll also briefly introduce a diagnostic tool called Tensorboard, which will be helpful for examining the progress of the training procedure of your model.

In [2]:
import numpy as np
import tensorflow as tf

## Dataflow graphs

In Tensorflow, everything lives in a graph. The neural networks were introduced to you via a graphical representation as a good way of thinking about them conceptually, and this perspective is beneficial from a computational perspective too. However, this means we'll need to write our scripts/programs in a slightly different way then what you'll be used to using vanilla Python.

A **computational graph** is a series of TensorFlow operations arranged into a graph. The graph is composed of two types of objects.

- Operations: The nodes of the graph. Operations describe calculations that consume and produce tensors.
- Tensors: The edges in the graph. These represent the values that will flow through the graph. Most TensorFlow functions return `tf.Tensors`.

In [20]:
a = 2.0 * tf.random_normal(shape=[]) + 3.0
print(a)

Tensor("add_4:0", shape=(), dtype=float32)


## Eager Execution

**The eager execution is the default operating mode of Tensorflow 2.0. So far we have actually been doing everything in eager execution.** TensorFlow's eager execution is an imperative programming environment that evaluates operations immediately, without building graphs: operations return concrete values instead of constructing a computational graph to run later. The advantages of this approach are easier debugging of all computations, natural control flow using Python statements instead of graph control flow, and an intuitive interface. The downside of eager mode is the reduced performance since graph-level optimizations such as common subexpression elimination and constant-folding are no longer available.

While the eager execution mode is easier to code (like plain Python syntax), the graph mode still holds benefits in being portable and highly-performanced. We will not dive deeper into the graph mode in this turorial. If you are interested and wanted to know how to convert to graphical execution in Tensorflow 2.0 via functions, you may refer to [this](https://github.com/tensorflow/community/blob/master/rfcs/20180918-functions-not-sessions-20.md)

In [3]:
tf.executing_eagerly()

True

Instead of defining a graph and then selecting the subgraph to execute at `sess.run()` time, the exact computation of interest is encapsulated in a Python callable. For example, the program above that uses `sess.run()` to compute `z0` and `z1` can be written as:

In [18]:
@tf.function
def compute_z1(x, y):
  return tf.add(x, y)

@tf.function
def compute_z0(x):
  return compute_z1(x, tf.square(x))

z0 = compute_z0(2.)
z1 = compute_z1(2., 2.)

AttributeError: module 'tensorflow' has no attribute 'function'

Where `tf.function` is a decorator that "defines a TensorFlow function". **A "TensorFlow function" defines a computation as a graph of TensorFlow operations, with named arguments and explicit return values**. Users define the function they want TensorFlow to "accelerate" as a Python function and integrate it into their Python program like any other Python function call.

## Basics of Tensorflow: Tensors

Conceptually, a Tensor is a multi-dimensional data container for modern machine learning. Similar to NumPy ndarray objects, tf.Tensor objects have a data type and a shape. The main difference is that tf.Tensors, additionally, can reside in accelerator memory (like a GPU). TensorFlow offers a rich library of operations (tf.add, tf.matmul, tf.linalg.inv etc.) that consume and produce tf.Tensors.

Tensors are characterized by two properties:
* datatype - this is usually tf.float32 or tf.int32
* shape - this could be e.g $10 \times 4$, $10 \times 20 \times 30$

As the image is $100$ pixels by $100$ pixels, we can store this as a $100 \times 100 \times 3$ tensor:

* the first two dimensions of the array correspond to a $(x, y)$ coordinate of a pixel;
* the last dimension corresponds to the Red, Green and Blue channels of the image.

In terms of the datatype, this could be either tf.int32 (for example, if the image is 24 bit - so there are 8 bits per channel - then the entries will be an integer between $0$ and $255 = 2^8 - 1$) or a tf.float32 (if we have images of different bit-depths, we will want to normalize them to lie in a range $[0, 1]$ so that our model is agnostic to this).

Reference:
* https://hackernoon.com/learning-ai-if-you-suck-at-math-p4-tensors-illustrated-with-cats-27f0002c9b32
* https://www.tensorflow.org/tutorials/customization/basics

Important types of tensors
In Tensorflow, there are several types of tensors which you will come across. The two main ones to keep in mind are:

* `tf.Variable` - Variables can be assigned values, and importantly these will be remembered across several calls to session.run. This will be used for the weights and biases in the network. (Why? If you think of one call of session.run as corresponding to one step of the optimization algorithm used to train your neural network, then you'd like to remember the weights/biases after each step!)
* `tf.placeholder` - Placeholders are given values only at runtime (when session.run is called), which will allow us to pass arguments to the Tensorflow operations we are about to use. For example, we can then use this to pass the training data into our model so we can then try and fit it.

## Tensorflow VS Numpy

**tf.Tensors work congruently with Numpy arrays (in eager execution)**. NumPy operations accept tf.Tensor arguments. TensorFlow math operations convert Python objects and NumPy arrays to tf.Tensor objects. The tf.Tensor.numpy method returns the object's value as a NumPy ndarray.

In [23]:
a = tf.constant([[1, 2], [3, 4]])
print('Tensor:', a)
print('Tensor shape:', a.shape)
print('Tensor type:', a.dtype)

Tensor: tf.Tensor(
[[1 2]
 [3 4]], shape=(2, 2), dtype=int32)
Tensor shape: (2, 2)
Tensor type: <dtype: 'int32'>


In [25]:
# Broadcasting support
print(tf.add(1, 2))
print(tf.add([1, 2], [3, 4]))
print(tf.square(5))
print(tf.reduce_sum([1, 2, 3]))

# Operator overloading is also supported
print(tf.square(2) + tf.square(3))

tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor([4 6], shape=(2,), dtype=int32)
tf.Tensor(25, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(13, shape=(), dtype=int32)


As seen from previous examples, tensors and Numpy arrays work congruently with each other. **We will show that we can also initialize a tensor using Numpy.** The function to be used is `tf.convert_to_tensor`

In [10]:
# create a Python array:
array_1d = np.array([1.3, 1, 4.0, 23.5])
tf_tensor = tf.convert_to_tensor(value=array_1d, dtype=tf.float64)

print(tf_tensor)
print('0:',tf_tensor[0])
print('2:',tf_tensor[2])

tf.Tensor([ 1.3  1.   4.  23.5], shape=(4,), dtype=float64)
0: tf.Tensor(1.3, shape=(), dtype=float64)
2: tf.Tensor(4.0, shape=(), dtype=float64)


## Constant/Variable

Variables is a primitive class in Tensorflow. It has the following properties:

* initial value is required
* updated during training
* in-memory buffer (saved/restored from disk)
* can be shared in a distributed environment
* they hold learned parameters of a model

In [28]:
# Let's try to compare with tf.Variable
a = tf.Variable([[1, 2], [3, 4]])
b = tf.Variable([[2, 2], [3, 3]])

print('Tensorflow variable:')
print(a)
print('Numpy variable:')
print(a.numpy())

# tf.multiply of two tf.Variable's 
print('Tensorflow multiplication:')
c_tf = tf.multiply(a, b)
print(c_tf)

print('Numpy muliplication:')
c_np = np.multiply(a, b)
print(c_np)

Tensorflow variable:
<tf.Variable 'Variable:0' shape=(2, 2) dtype=int32, numpy=
array([[1, 2],
       [3, 4]])>
Numpy variable:
[[1 2]
 [3 4]]
Tensorflow multiplication:
tf.Tensor(
[[ 2  4]
 [ 9 12]], shape=(2, 2), dtype=int32)
Numpy muliplication:
tf.Tensor(
[[ 2  4]
 [ 9 12]], shape=(2, 2), dtype=int32)
tf.Tensor(
[[ 2  4]
 [ 9 12]], shape=(2, 2), dtype=int32)


In [31]:
# Don't do this! yields weird answers
d = tf.add(a,1)
d_np = np.multiply(a, d)
print(d_np)

[[<tf.Tensor: id=243, shape=(2, 2), dtype=int32, numpy=
array([[2, 4],
       [6, 8]])>
  <tf.Tensor: id=246, shape=(2, 2), dtype=int32, numpy=
array([[ 3,  6],
       [ 9, 12]])>]
 [<tf.Tensor: id=249, shape=(2, 2), dtype=int32, numpy=
array([[ 4,  8],
       [12, 16]])>
  <tf.Tensor: id=252, shape=(2, 2), dtype=int32, numpy=
array([[ 5, 10],
       [15, 20]])>]]


## Keras and Tensorboard¶
One big change in Tensorflow 2.0 is the integration of Keras as a higher level API. The integration makes it easier and faster for new users to get started with TensorFlow. Below we will quickly go through a classification example provided in this post using tf.keras. Then we will implement another model using lower-level Tensorflow without Keras, just to give you a taste.

**Quick guide for Tensorboard**

Tensorboard is a powerful tool provided by TensorFlow. It allows developers to check their graph and trend of parameters. This guide will give you a basic under standing on how to set up Tensorboard graph in your code, start tensorboard on your local machine/GCP instance and how to access the interface.

For complete instructions, check the official guide on Tensorflow web site [here](https://www.tensorflow.org/get_started/summaries_and_tensorboard).

**How to start tensorboard**

**Local**

To start your Tensorboard on your local machine, you need to specify a log directory for the service to fetch the graph. For example, in your command line, type:

```shell
$ tensorboard --logdir="~/log"
```

Then, Tensorboard will start running. By default, it will be running on port 6006:

``` shell
TensorBoard 1.8.0 at http://localhost:6006 (Press CTRL+C to quit)
```

Make sure Tensorboard is running, you can visit http://localhost:6006 In your browser and you should be able to see the main page of Tensorboard. If the page is shown as below, it means Tensorboard is running correctly. The report is due to lack of event file, but we can just leave it there for now.

![Tensorboard](./notebook_images/tensorboard.png)

**GCP**

To set up the Tensorboard on GCP is the same as above. However, we're not able to check the Tensorboard UI directly through our browser. In order to visit the page through our local browser, we should link the port of our local machine to the port on GCP. It is similar to what we did previously for Jupyter Notebook.

In the command line on your local machine, type:

```shell
$ gcloud compute ssh --ssh-flag="-L 9999:localhost:9999 -L 9998:localhost:6006" "ecbm4040@YOUR_INSTANCE"
```

 This will bind your port of your local machine to the port on GCP instance. In this case, your local port 9999 is binded with 9999 on GCP, while local port 9998 is binded with 6006 on GCP. You can change whatever port you like as long as it does not confilct with your local services.

After connecting to GCP using the command, you will be able to see the result page.


**Export Tensorboard events into log directory**

To generate data files for Tensorboard, we should use class `tf.summary.FileWriter`. This class will save your network graph sturcuture and all the variable summary. The file writer will save the graph and the summary into a directory based on the current timestamp. Here is the code snippet:

```python
cur_model_name = 'lenet_{}'.format(int(time.time()))
# ...

# set up summary writer for tensorboard
merge = tf.summary.merge_all()	# merge all the summary for variables for execution
writer = tf.summary.FileWriter("log/{}".format(cur_model_name), sess.graph)
```

The following code will save all the parameter summary and marked with iteration_total. These data will be displayed in the Tensorboard later on.

```python
# ... previous code ...
# ...
				if iter_total % 100 == 0:
                    # do validation
                    valid_eve, merge_result = sess.run([eve, merge], feed_dict={xs: X_val, ys: y_val})
                    valid_acc = 100 - valid_eve * 100 / y_val.shape[0]
                    if verbose:
                        print('{}/{} loss: {} validation accuracy : {}%'.format(
                            batch_size * (itr + 1),
                            X_train.shape[0],
                            cur_loss,
                            valid_acc))

                    # save the merge result summary
                    writer.add_summary(merge_result, iter_total)
```